### Code to process the downloaded NO2 data 

In [1]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import numpy as np
import xesmf as xe
import glob 
import os

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

# load additional function from LTPY
%run ../LTPY_functions.ipynb

In [2]:
def subset_tropomi_data(DATADIR, lat_resn, lon_resn, lat_min, lat_max, lon_min, lon_max, var_name):
    # A function that reads in all Tropomi files in a directory and subsets and regrids them to the required lat and lon extents, 
    # and for the defined pixel resolution
    # example usage: subset_tropomi_data(DATADIR, 5.5, 7.0, lat_min, lat_max, lon_min, lon_max)
    
    # creating a new grid that all timesteps of the irregular grid will be interpolated to
    # Calculating the new grid spacings
    lat_num = calculate_grid_spacing(lat_min, lat_max, lat_resn)
    lon_num = calculate_grid_spacing(lon_min, lon_max, lon_resn)

    # creating a 7.0 x 5.5 km grid (or any defined lon_resn x lat_resn)
    output_latitudes = np.linspace(lat_min,lat_max, lat_num)
    output_longitudes = np.linspace(lon_min, lon_max, lon_num)
    
    
    # defining the output grid for the xarray regridding step
    output_grid = xr.Dataset({'lat': (['lat'], output_latitudes), 'lon': (['lon'], output_longitudes)})
    
    # an array of all the subset and regridded Tropomi data
    ds_subset_all = []
    counter = 0 
    for filename in glob.glob(DATADIR + 'S5P*'):
        #print the progress periodically when testing
        if counter % 10 == 0:
            print (counter, filename)
        
        #print (counter, filename)
        
        # opening the files one by one (it crashes if trying to add them all to the same array due to the large file size)
        ds = xr.open_mfdataset(filename, combine='nested', group='PRODUCT')

        # subsetting the data
        ds_subset0 = generate_geographical_subset(xarray=ds, 
                                                 latmin=lat_min, 
                                                 latmax=lat_max, 
                                                 lonmin=lon_min, 
                                                 lonmax=lon_max)
        

        # assignment of latitude and longitude is required for the HCHO data, but not for the CH4 data, not sure why as they seem to have the same format. 
        ds_subset0 = ds_subset0.set_coords(('latitude','longitude'))
        
        # load the subset
        ds_subset = ds_subset0[var_name].load()
        
        # some files have no timesteps so the regridding step doesn't work. Only regridding if there are more than 0 timesteps.
        if len(ds_subset > 0):

            # using xesmf function to calculate the regridding matrix using nearest neighbour method. Periodic = False defined for irregular grid.
            # need to run the precision product also and use it to mask the data (0.5 for HCHO?)
            regridder = xe.Regridder(ds_subset[0], output_grid, 'nearest_s2d', periodic=False)

            # regridding the data
            ds_new = regridder(ds_subset)

            # appending the regridded dataset to an array (this will later be concatenated - note that the dates are not currently in the right order)
            ds_subset_all.append(ds_new)
        
        counter = counter + 1
        
    return (ds_subset_all)
    

In [3]:
# Calculating the new grid spacings. Using 7km across track and 5.5km along track as this is the highest resolution for the scan direction (but not representative of all the data)

def calculate_grid_spacing(l_min, l_max, res):
    # returns the number of lats/lons to be included in each dimension of the new grid
    # usage example: calculate_grid_spacing(latitudes, 5.5)
    # where array is the range of lats/lons in degrees
    # and res is the required resolution of the output grid
    
    # difference between the maximum and min lat/lon values of the range
    l_diff = l_max-l_min
    
    # range in km, 1 degree ~ 111.139 km at the equator
    l_dist = l_diff*111.139  
    
    # number of pixels with specified resolution that will fit into this extent
    # rounding to the nearest number
    num = int(round(l_dist/res,0))
    
    return num

In [4]:
def finalise_outputs(ds, var_name, out_file, ddir):
    # concatenating the array of xarrays, these are not currently in the right date order, so also sorting them by date
    ds_final = xr.concat(ds, dim='time')
    
    # sorting into ascending date range
    ds_sorted = ds_final.sortby('time', ascending=True)
    
    # assigning new var name
    ds_new = ds_sorted.to_dataset(name = var_name)
    
    print ('saving to file: ' + out_file)
    ds_new.to_netcdf(ddir + out_file)
    
    return ds_new

In [5]:
def remove_repeated_timesteps(ds, var_name):
    # remove repeated time steps in an xarray dataset
    # example usage:  ds_methane_new = remove_repeated_timesteps(ds_methane, 'Methane (ppb)')
    # This is currently used in the next code,Analyse_L2_Tropomi_CH4.ipynb, but could be done here instead
    
    da = ds[var_name].copy()

    # first remove all dates with no data at all
    da = da.dropna(dim='time', how='all')

    print ('Empty time steps (these have been removed): ' + str(len(ds[var_name]) - len(da)))

    for i in range(len(da)-1):
        # checking if the timestep is repeated
        if da[i].time.values == da[i+1].time.values:
            #print ('Matched at: ' + str(i) + ' and ' + str(ds[i].time.values))

            # merge the data from these two dates
            da1 = da[i]
            da2 = da[i+1]
            #print (ds1.time.values, ds2.time.values)
            ds = xr.merge([da1,da2])
            da[i] = ds[var_name]
        
    # drop duplicates along the time dimension, keeping the first date which contains the merged information
    _, index = np.unique(da['time'], return_index=True)
    da2 = da.isel(time=index)
    ds_new = da2.to_dataset(name = var_name)
    ds_new

    return ds_new

### Covering the full Cuvette Centrale region
lat_min = -5.0
lat_max = 3.5
lon_min = 15.0
lon_max = 23.0

Sentinel-5P TROPOMI variables have the following dimensions:
- corner: pixel corner index
- ground_pixel: the number of spectra in a measurement / across-track dimension index
- layer: this dimension indicates the vertical grid of profile variables
- scanline: the number of measurements in the granule / along-track dimension index
- time: time reference for the data

In [7]:
# Extent of the Cuvette Centrale peatlands
lat_min = -5.0
lat_max = 3.5
lon_min = 15.0
lon_max = 23.0

DATADIR = '/exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC_RPRO/'
DATADIR_OFFL = '/exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/'
DATADIR2 = '/exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/gridded/'

In [ ]:
%%time
# using a higher resolution grid than the base Tropomi resolution (approx 1 x 1 km) to account for pixels not being exactly aligned between timesteps
# running seperately for each year, month, day folder (when using the offline data to fill gaps)
#y= '2021'
#m = '01'
#d = '07'

#DATADIR = HCHO_dir + y + '/' + m + '/' + d + '/'

DATADIR = '/exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC_RPRO/'
DATADIR_OFFL = '/exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/'
DATADIR2 = '/exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/gridded/'
print (DATADIR)
ds_all = subset_tropomi_data(DATADIR, 1.0, 1.0, lat_min, lat_max, lon_min, lon_max, 'nitrogendioxide_tropospheric_column')


ds_all
finalise_outputs(ds_all, 'NO2 (mol m-2)', 'NO2_OFFL_ts_1km.nc', DATADIR2)

# next steps are in the code (combining dates, and final analysis): Analyse_L2_Tropomi_HCHO.ipynb

In [ ]:
# precision data RPRO
ds_prec = subset_tropomi_data(DATADIR, 1.0, 1.0, lat_min, lat_max, lon_min, lon_max, 'nitrogendioxide_tropospheric_column_precision')
finalise_outputs(ds_prec, 'NO2 (mol m-2)', 'NO2_RPRO_precision_ts_1km.nc', DATADIR2)

In [10]:
# precision data OFFL
ds_prec = subset_tropomi_data(DATADIR_OFFL, 1.0, 1.0, lat_min, lat_max, lon_min, lon_max, 'nitrogendioxide_tropospheric_column_precision')
finalise_outputs(ds_prec, 'NO2 (mol m-2)', 'NO2_OFFL_precision_ts_1km.nc', DATADIR2)

0 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20220929T112040_20220929T130209_25709_03_020400_20221001T034542.nc
10 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230505T114148_20230505T132318_28802_03_020500_20230507T040617.nc
20 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230305T121800_20230305T135930_27937_03_020400_20230307T044832.nc
30 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20221227T114718_20221227T132848_26972_03_020400_20221230T081318.nc
40 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230430T113456_20230430T131626_28731_03_020500_20230502T040529.nc
50 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230403T113735_20230403T131906_28348_03_020500_20230405T035

<xarray.Dataset>
Dimensions:        (lat: 945, lon: 889, time: 421)
Coordinates:
  * time           (time) datetime64[ns] 2022-07-26 2022-07-27 ... 2023-05-28
  * lon            (lon) float64 15.0 15.01 15.02 15.03 ... 22.98 22.99 23.0
  * lat            (lat) float64 -5.0 -4.991 -4.982 -4.973 ... 3.482 3.491 3.5
Data variables:
    NO2 (mol m-2)  (time, lat, lon) float64 1.604e-05 nan ... 1.202e-05

In [ ]:
# qa value for masking RPRO
ds_qa = subset_tropomi_data(DATADIR, 1.0, 1.0, lat_min, lat_max, lon_min, lon_max, 'qa_value')
finalise_outputs(ds_qa, 'qa value', 'NO2_RPRO_qa_value_ts_1km.nc', DATADIR2)

In [9]:
# qa value for masking OFFL
ds_qa = subset_tropomi_data(DATADIR_OFFL, 1.0, 1.0, lat_min, lat_max, lon_min, lon_max, 'qa_value')
finalise_outputs(ds_qa, 'qa value', 'NO2_OFFL_qa_value_ts_1km.nc', DATADIR2)

0 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20220929T112040_20220929T130209_25709_03_020400_20221001T034542.nc
10 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230505T114148_20230505T132318_28802_03_020500_20230507T040617.nc
20 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230305T121800_20230305T135930_27937_03_020400_20230307T044832.nc
30 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20221227T114718_20221227T132848_26972_03_020400_20221230T081318.nc
40 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230430T113456_20230430T131626_28731_03_020500_20230502T040529.nc
50 /exports/geos.ed.ac.uk/palmer_group/managed/s0677837/Peatlands/TROPOMI/NO2/CC/S5P_OFFL_L2__NO2____20230403T113735_20230403T131906_28348_03_020500_20230405T035

<xarray.Dataset>
Dimensions:   (lat: 945, lon: 889, time: 421)
Coordinates:
  * time      (time) datetime64[ns] 2022-07-26 2022-07-27 ... 2023-05-28
  * lon       (lon) float64 15.0 15.01 15.02 15.03 ... 22.97 22.98 22.99 23.0
  * lat       (lat) float64 -5.0 -4.991 -4.982 -4.973 ... 3.473 3.482 3.491 3.5
Data variables:
    qa value  (time, lat, lon) float64 1.0 nan nan nan ... 0.74 0.74 0.74 0.74